In [181]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import linregress

df = pd.read_csv("../cleandata/athletes_and_medalcount_noNaN")
df1 = pd.read_csv("../cleandata/athletes_and_medalcount")

In [182]:
df = df[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team',
       'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'region_x',
       'notes_x', 'Bronze', 'Gold', 'Silver', 'n/a']]
df["Medals"] = df['Bronze'] + df['Silver']+ df['Gold']
df["Attempts"] = df['Bronze'] + df['Silver']+ df['Gold']+ df['n/a']

In [183]:
df1 = df1[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team',
       'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'region_x',
       'notes_x', 'Bronze', 'Gold', 'Silver', 'n/a']]
df1["Medals"] = df1['Bronze'] + df1['Silver']+ df1['Gold']
df1["Attempts"] = df1['Bronze'] + df1['Silver']+ df1['Gold']+ df1['n/a']

In [188]:
sports_list = df1["Sport"].unique()
r_weight = []
r_height = []
medal_rate = []
num_of_events = []
games = []
competitors = []

In [189]:
for sport in sports_list:
    try:
        df_loop = df
        df_loop = df_loop.loc[df_loop['Sport'] == f'{sport}']
        df_loop = df_loop.groupby(by = 'ID').agg({
            "Height" : "max",
            "Weight" : "max",
            "Medals" : "sum",
            "Attempts" : "sum"
        })
        try:
            df_loop["Medal Rate"] = df_loop["Medals"] / df_loop["Attempts"] *100
        except:
            0
        plt.clf()
        df_loop = df_loop.loc[df_loop["Medals"] >= 1]
        x_val = df_loop["Height"]
        y_val = df_loop["Medal Rate"]
        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_val, y_val)
        regress_values = x_val * slope + intercept
        plt.scatter(x_val,y_val)
        plt.plot(x_val,regress_values,"r-")
        line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
        plt.annotate(line_eq,(5,5), fontsize=25,color="red")
        plt.xlabel('Height')
        plt.ylabel('Medal Rate')
        plt.title("Height vs Medal Rate")
        plt.savefig(f'plot_outputs/{sport}xheight.jpeg')
        r_height.append(rvalue)
        plt.clf()
        x_val = df_loop["Weight"]
        y_val = df_loop["Medal Rate"]
        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_val, y_val)
        regress_values = x_val * slope + intercept
        plt.scatter(x_val,y_val)
        plt.plot(x_val,regress_values,"r-")
        line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
        plt.annotate(line_eq,(5,5), fontsize=25,color="red")
        plt.xlabel('Weight')
        plt.ylabel('Medal Rate')
        plt.title("Weight vs Medal Rate")
        plt.savefig(f'plot_outputs/{sport}xweight.jpeg')
        r_weight.append(rvalue)
        plt.clf()
    except:
        r_height.append(0)
        r_weight.append(0)

    try:
        df_loop_sport = df1
        df_loop_sport = df_loop_sport.loc[df_loop_sport['Sport'] == f'{sport}']
        df_loop_sport = df_loop_sport.agg({
                    "Medals" : "sum",
                    "Attempts" : "sum",
                    "Event" : "nunique",
                    "Games" : "nunique",
                    "ID" : "nunique"
                })        
        try:
            medal_rate.append(df_loop_sport["Medals"] / df_loop_sport["Attempts"] *100)

            
        except:
            0
        games.append(df_loop_sport["Games"])
        competitors.append(df_loop_sport["ID"])
        num_of_events.append(df_loop_sport["Event"])
    except:
        medal_rate.append(0)
        num_of_events.append(0)
        games.append(0)
        competitors.append(0)


<Figure size 432x288 with 0 Axes>

In [190]:
print(len(sports_list))
print(len(r_height))
print(len(r_weight))
print(len(medal_rate))
print(len(num_of_events))


66
66
66
66
66


In [191]:
sport_stats = pd.DataFrame({
    "Sport" : sports_list,
    "R2 Weight": r_weight,
    "R2 Height" : r_height,
    "Medal Rate" : medal_rate,
    "Number of Events" : num_of_events,
    "# of Games Appeared" : games,
    "Olympians Competed": competitors
        })
sport_stats = sport_stats.sort_values("Medal Rate", ascending = False)
sport_stats.head(20)

,Sport,R2 Weight,R2 Height,Medal Rate,Number of Events,# of Games Appeared,Olympians Competed
65,Aeronautics,0.000000,0.000000,100.000000,1,1,1
63,Alpinism,0.000000,0.000000,100.000000,1,3,25
51,Rugby,0.000000,0.000000,100.000000,1,4,155
54,Lacrosse,0.000000,0.000000,100.000000,1,2,60
56,Cricket,0.000000,0.000000,100.000000,1,1,24
64,Basque Pelota,0.000000,0.000000,100.000000,1,1,2
57,Racquets,0.000000,0.000000,83.333333,2,1,7
62,Roque,0.000000,0.000000,75.000000,1,1,4
55,Polo,0.000000,0.000000,70.526316,1,5,87
3,Tug-Of-War,0.000000,0.000000,67.647059,1,6,160


In [192]:
sport_stats.to_csv("Sport_lvl_Stats.csv")